In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from scipy.sparse import csc_matrix,diags
from scipy.sparse.linalg import spsolve,inv
from scipy.linalg import det
from IPython.display import display, HTML,IFrame

interactive = True
use_cholesky = True
filename = 'woody-hi'
html=True

In [2]:
if interactive:
    html=False
if html:
    mp.website()
file_counter = 0
def htmlit(plot,method=1):
    if method==1:
        global file_counter
        unique_filename = f"./htmls/plot_{filename}{file_counter}.html"
        file_counter += 1
        plot.save(unique_filename)
        return unique_filename
    else:
        html = plot.to_html(imports=True, html_frame=False)
        return HTML(html)

In [3]:
v, f = igl.read_triangle_mesh(f'data/{filename}.off')
labels = np.load(f'data/{filename}.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

In [4]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    v_deformed = add_detail(v_deformed)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
#pos_f.deformer = lambda x:x

In [5]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [6]:
slicesconstraint = np.where(labels>0)[0]
slicesfree=np.where(labels==0)[0]
mass_mat=igl.massmatrix(v,f,igl.MASSMATRIX_TYPE_BARYCENTRIC)
M=mass_mat
cot_mat=csc_matrix(igl.cotmatrix(v,f))
A=cot_mat@inv(M)@cot_mat
#b=np.zeros_like(target_pos)
#b = csc_matrix((target_pos.shape[0], target_pos.shape[1]), dtype=np.float64)
Aff=A[slicesfree,:][:,slicesfree]
Afc=A[slicesfree,:][:,slicesconstraint]

def position_deformer(target_pos):
    b=-Afc@target_pos[slicesconstraint]
    x = spsolve(Aff,b)
    new=target_pos.copy()
    new[slicesfree]=x
    return new

pos_f.deformer = position_deformer
B_v=position_deformer(handle_vertex_positions)


## Below for step 3
d=v-B_v
normal_B=igl.per_vertex_normals(B_v, f)
#normal_B=normal_B/np.linalg.norm(normal_B,axis=1)[:,None]
adjlist=igl.adjacency_list(f)
x_i=np.zeros_like(normal_B) # v*3
y_i=np.zeros_like(normal_B)
longest_adj=np.zeros((v.shape[0]),dtype=int)
for i in range(v.shape[0]):
    edgelist=B_v[adjlist[i]]-B_v[i]
    projectlist=edgelist-np.outer(np.dot(edgelist,normal_B[i]) , normal_B[i]) # the projected edges
    longest_adj[i]=np.argmax(np.linalg.norm(projectlist,axis=1)) # the index inside a adjlist, usually 0-6
    x_i[i]= projectlist[longest_adj[i]] # the longest one
    x_i[i]=x_i[i]/np.linalg.norm(x_i[i])
    y_i[i]=np.cross(x_i[i], normal_B[i])

d_x=np.einsum('ij,ij->i', x_i, d)
d_y=np.einsum('ij,ij->i', y_i, d)
d_z=np.einsum('ij,ij->i', normal_B, d)

print(d_x)
print(d_y)
def add_detail(B_new):
    new_normals=igl.per_vertex_normals(B_new,f)
    #new_normals=new_normals/np.linalg.norm(new_normals,axis=1)[:,None]
    x_j=np.zeros_like(new_normals)
    y_j=np.zeros_like(new_normals)
    for i in range(v.shape[0]):
        x_j[i]=B_new[adjlist[i][longest_adj[i]]]-B_new[i]
        x_j[i]=x_j[i]-np.dot(x_j[i],new_normals[i])*new_normals[i]
        x_j[i]=x_j[i]/np.linalg.norm(x_j[i])
        y_j[i]=np.cross(x_j[i],new_normals[i])
    new=B_new.copy()
    offset=np.zeros_like(B_new)
    offset+=d_x[:,None]*x_j
    offset+=d_y[:,None]*y_j
    offset+=d_z[:,None]*new_normals
    new+=offset
    return new

[ 0.00322185 -0.00874238  0.00094199 ... -0.00584843  0.00036549
  0.00467662]
[-0.04017827 -0.04029144 -0.03434543 ...  0.01803664  0.01211022
 -0.03438885]


In [7]:
## result for step 1
p = mp.plot(v, f, c=labels)
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_cactus0.html.


In [8]:
## result for step 1
p = mp.plot(B_v, f, c=labels)
if html:
    html=htmlit(p)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_cactus1.html.


In [9]:
from sksparse.cholmod import cholesky
factor = cholesky(Aff)
def Cholesky_deformer(target_pos):
    b=-Afc@target_pos[slicesconstraint]
    x = factor(b)
    new=target_pos.copy()
    new[slicesfree]=x
    return new

if use_cholesky:
    pos_f.deformer=Cholesky_deformer

In [10]:
## Widget UI
if interactive==True:
    p = mp.plot(handle_vertex_positions, f, c=labels)
    iw.interact(pos_f, **widgets_wrapper())

In [11]:
def manual_pos_f(verts, labels, s, x, y, z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    verts[slices] = r.apply(v_slice - center) + center
    v_smoothed = pos_f.deformer(verts) # B'
    v_deformed = add_detail(v_smoothed) # S'
    return v_smoothed,v_deformed
    
if interactive==False:
    c = np.load(f'data/{filename}.pos.npy')
    V=v
    #B,_=manual_pos_f(V,labels,0,0,0,0,0,0)
    B_prime=V
    for i in range(c.shape[0]):
        B_prime,S_prime=manual_pos_f(V,labels,c[i,0].astype(int),c[i,1],c[i,2],c[i,3],c[i,4],c[i,5],c[i,6])
    p1= mp.plot(B_prime, f, c=labels)
    p2= mp.plot(S_prime, f, c=labels)

In [12]:
if html and interactive==False:
    html=htmlit(p1)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_cactus2.html.


In [13]:
if html and interactive==False:
    html=htmlit(p2)
IFrame(src=html, width=700, height=600)

Plot saved to file ./htmls/plot_cactus3.html.
